In [23]:
# Import dependencies
from requests_html import HTMLSession, AsyncHTMLSession
import pandas as pd
session = AsyncHTMLSession()


In [24]:
# Get page
r = await session.get("https://www.ourlads.com/nfldepthcharts/depthcharts.aspx")
await r.html.arender()

In [25]:
element = r.html.find('#ctl00_phContent_gvChart')[0].html

In [110]:
# Convert to df
df = pd.read_html(element)[0]
# Filter for only the needed columns
df = df[['Team', 'Pos', 'Player 1', 'Player 2', 'Player 3', 'Player 4', 'Player 5']]
# Rename columns of Position Ranks; limit number of ranks to three
df = df.rename(columns={
    'Player 1':'1',
    'Player 2':'2',
    'Player 3':'3',
    'Player 4':'3',
    'Player 5':'3',
})
# Filter only relevant positions
posList = ['LWR', 'RWR', 'SWR', 'TE', 'QB', 'RB', 'PK', 'PR', 'KR', 'RES']
df = df.loc[df['Pos'].isin(posList)]

# Transpose columns to rows to get position ranks in row form rather than column
df = df.melt(id_vars=["Team", "Pos"], 
    var_name="posRank", 
    value_name="playerName")
    # Rename columns to match MyFantasyLeague
df = df.rename(columns={'Team':'team', 'Pos':'pos'})
# Create id_ourlads column
df = df.dropna(subset='playerName')
df['lName'] = df['playerName'].str.split(", ", expand=True)[0]
df['fName'] = df['playerName'].str.split(", ", expand=True)[1].str.split(" ", expand=True)[0]
df['id_ourlads'] = df['fName'] + " " + df['lName']
df['id_ourlads'] = df['id_ourlads'].str.replace(".", "")
df['id_ourlads'] = df['id_ourlads'].str.title()
df['id_ourlads'] = [" ".join(x.split(" ")[:2]) for x in df['id_ourlads']]

# Highlight PR & KRs
cond1 = df['pos']=='PR'
cond2 = df['pos']=='KR'
cond3 = df['posRank']=="1"
prs = df.loc[cond1 & cond3]['id_ourlads'].unique()
krs = df.loc[cond2 & cond3]['id_ourlads'].unique()
df.loc[df['id_ourlads'].isin(prs), 'PR'] = True
df.loc[df['id_ourlads'].isin(krs), 'KR'] = True

# Reduce
df = df.loc[(df['PR']==True) | (df['KR']==True)]
df = df.drop_duplicates(subset='id_ourlads', ignore_index=True)
df = df[['id_ourlads', 'PR', 'KR']]


In [111]:
# Merge in id_dict
ids = pd.read_csv('data_sharks/id_dict.csv', dtype={'player_id':str, 'id_player_sharks':str})
ids = ids[['id_ourlads', 'player_id']]
df = df.merge(ids, how='left', on='id_ourlads')
df

,id_ourlads,PR,KR,player_id
0,Greg Dortch,True,True,5970
1,Dee Alford,True,NaN,NaN
2,Cordarrelle Patterson,NaN,True,1535
3,Devin Duvernay,True,True,6847
4,Deonte Harty,True,True,6234
5,Raheem Blackshear,True,True,8255
6,Trent Taylor,True,NaN,4218
7,Tyler Scott,NaN,True,9490
8,Charlie Jones,True,True,10228
9,Donovan Peoples-Jones,True,NaN,6824


In [112]:
df.to_csv('data_sharks/prkr.csv', index=False)